In [151]:
!pip install xgboost

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [206]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import SelectKBest , f_classif
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [207]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [154]:
data = pd.read_csv('train.csv')

In [155]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [156]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [157]:
data['PassengerId'].nunique()

8693

In [158]:
data.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,217
ShoppingMall,208
VIP,203
HomePlanet,201
Name,200
Cabin,199
VRDeck,188
Spa,183
FoodCourt,183
Destination,182


In [159]:
data.drop('Name',axis=1,inplace=True)

In [160]:
data['PassengerGroup'] = data['PassengerId'].str.split('_').str[0]

In [161]:
data['PassengerGroup'].nunique()

6217

In [162]:
data['Destination'].nunique()

3

In [163]:
data['Destination'].value_counts()

,count
Destination,
TRAPPIST-1e,5915
55 Cancri e,1800
PSO J318.5-22,796


In [164]:
data['HomePlanet'].nunique()

3

In [165]:
data['HomePlanet'].value_counts()

,count
HomePlanet,
Earth,4602
Europa,2131
Mars,1759


In [166]:
copy = data.copy()

In [167]:
group_dest = copy.groupby('PassengerGroup')['Destination'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

copy['Destination'] = copy.apply(
        lambda row: group_dest[row['PassengerGroup']] if pd.isna(row['Destination']) else row['Destination'],
        axis=1
    )

In [168]:
copy['Destination'].fillna(copy['Destination'].mode()[0], inplace=True)

In [169]:
copy.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,217
ShoppingMall,208
VIP,203
HomePlanet,201
Cabin,199
VRDeck,188
Spa,183
FoodCourt,183
RoomService,181
Age,179


In [170]:
parts = copy['Cabin'].str.split('/', expand=True)
copy['Cabin_Deck'] = parts[0]
copy['Cabin_Num'] = parts[1]
copy['Cabin_Side'] = parts[2]

In [171]:
copy['Cabin_Num'] = pd.to_numeric(copy['Cabin_Num'], errors='coerce')

In [172]:
for col in ['Cabin_Deck', 'Cabin_Side']:
      group_fill = copy.groupby('PassengerGroup')[col].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
      copy[col] = copy.apply(lambda row: group_fill[row['PassengerGroup']] if pd.isna(row[col]) else row[col], axis=1)

group_num = copy.groupby('PassengerGroup')['Cabin_Num'].median()
copy['Cabin_Num'] = copy.apply(lambda row: group_num[row['PassengerGroup']] if pd.isna(row['Cabin_Num']) else row['Cabin_Num'], axis=1)


In [173]:
copy['Cabin_Deck'].fillna(copy['Cabin_Deck'].mode()[0], inplace=True)
copy['Cabin_Side'].fillna(copy['Cabin_Side'].mode()[0], inplace=True)
copy['Cabin_Num'].fillna(copy['Cabin_Num'].median(), inplace=True)

In [174]:
copy.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,217
ShoppingMall,208
VIP,203
HomePlanet,201
Cabin,199
VRDeck,188
FoodCourt,183
Spa,183
RoomService,181
Age,179


In [175]:
group_home = copy.groupby('PassengerGroup')['HomePlanet'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

copy['HomePlanet'] = copy.apply(
        lambda row: group_home[row['PassengerGroup']] if pd.isna(row['HomePlanet']) else row['HomePlanet'],
        axis=1
    )

In [176]:
copy['HomePlanet'].fillna(copy['HomePlanet'].mode()[0], inplace=True)

In [177]:
copy.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,217
ShoppingMall,208
VIP,203
Cabin,199
VRDeck,188
FoodCourt,183
Spa,183
RoomService,181
Age,179
PassengerId,0


In [178]:
copy['Cabin_Deck'].value_counts()

,count
Cabin_Deck,
F,2909
G,2588
E,885
B,796
C,761
D,487
A,262
T,5


In [179]:
copy['Cabin_Side'].value_counts()

,count
Cabin_Side,
S,4442
P,4251


In [180]:
copy['VIP'].value_counts()

,count
VIP,
False,8291
True,199


In [181]:
copy['CryoSleep'].value_counts()

,count
CryoSleep,
False,5439
True,3037


In [182]:
copy.drop(['PassengerId','Cabin'],axis=1,inplace=True)

In [183]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   HomePlanet      8693 non-null   object 
 1   CryoSleep       8476 non-null   object 
 2   Destination     8693 non-null   object 
 3   Age             8514 non-null   float64
 4   VIP             8490 non-null   object 
 5   RoomService     8512 non-null   float64
 6   FoodCourt       8510 non-null   float64
 7   ShoppingMall    8485 non-null   float64
 8   Spa             8510 non-null   float64
 9   VRDeck          8505 non-null   float64
 10  Transported     8693 non-null   bool   
 11  PassengerGroup  8693 non-null   object 
 12  Cabin_Deck      8693 non-null   object 
 13  Cabin_Num       8693 non-null   float64
 14  Cabin_Side      8693 non-null   object 
dtypes: bool(1), float64(7), object(7)
memory usage: 959.4+ KB


In [184]:
copy.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,217
ShoppingMall,208
VIP,203
VRDeck,188
Spa,183
FoodCourt,183
RoomService,181
Age,179
Destination,0
HomePlanet,0


In [185]:
copy['VIP'].values

array([False, False, True, ..., False, False, False], dtype=object)

In [186]:
copy['VIP'] = copy['VIP'].astype('boolean')

In [187]:
copy['VIP'].dtype

BooleanDtype

In [188]:
copy['VIP'].value_counts()

,count
VIP,
False,8291
True,199


In [189]:
copy['CryoSleep'].values

array([False, False, False, ..., False, False, False], dtype=object)

In [190]:
copy['CryoSleep'] = copy['CryoSleep'].astype('boolean')

In [191]:
copy['CryoSleep'].dtype

BooleanDtype

In [192]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   HomePlanet      8693 non-null   object 
 1   CryoSleep       8476 non-null   boolean
 2   Destination     8693 non-null   object 
 3   Age             8514 non-null   float64
 4   VIP             8490 non-null   boolean
 5   RoomService     8512 non-null   float64
 6   FoodCourt       8510 non-null   float64
 7   ShoppingMall    8485 non-null   float64
 8   Spa             8510 non-null   float64
 9   VRDeck          8505 non-null   float64
 10  Transported     8693 non-null   bool   
 11  PassengerGroup  8693 non-null   object 
 12  Cabin_Deck      8693 non-null   object 
 13  Cabin_Num       8693 non-null   float64
 14  Cabin_Side      8693 non-null   object 
dtypes: bool(1), boolean(2), float64(7), object(5)
memory usage: 857.5+ KB


In [193]:
copy.drop(['PassengerGroup'],axis=1,inplace=True)

In [194]:
copy = pd.get_dummies(copy,columns=['HomePlanet','Destination','Cabin_Deck','Cabin_Side'],drop_first=True)

In [195]:
copy.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,217
ShoppingMall,208
VIP,203
VRDeck,188
Spa,183
FoodCourt,183
RoomService,181
Age,179
Transported,0
Cabin_Num,0


In [196]:
imputer = IterativeImputer(random_state=42)
copy_imputed = imputer.fit_transform(copy)
copy = pd.DataFrame(copy_imputed, columns=copy.columns)

In [197]:
copy.isnull().sum().sort_values(ascending=False)

,0
CryoSleep,0
Age,0
VIP,0
RoomService,0
FoodCourt,0
ShoppingMall,0
Spa,0
VRDeck,0
Transported,0
Cabin_Num,0


In [198]:
copy.duplicated().sum()

np.int64(11)

In [199]:
copy.drop_duplicates(inplace=True)

In [200]:
copy['Transported'].value_counts()

,count
Transported,
1.0,4367
0.0,4315


In [205]:
copy.corr()['Transported']

,Transported
CryoSleep,0.471092
Age,-0.074249
VIP,-0.037465
RoomService,-0.245974
FoodCourt,0.047547
ShoppingMall,0.010069
Spa,-0.222867
VRDeck,-0.209215
Transported,1.000000
Cabin_Num,-0.046147


In [208]:
x = copy.drop('Transported',axis=1)
y = copy['Transported']

In [209]:
selector = SelectKBest(score_func=f_classif, k=10)
x_selected = selector.fit_transform(x, y)
selected_features = x.columns[selector.get_support()]

In [210]:
selected_features

Index(['CryoSleep', 'RoomService', 'Spa', 'VRDeck', 'HomePlanet_Europa',
       'Destination_TRAPPIST-1e', 'Cabin_Deck_B', 'Cabin_Deck_C',
       'Cabin_Deck_E', 'Cabin_Side_S'],
      dtype='object')

In [211]:
x = x[selected_features]

In [212]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)

In [213]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [219]:
def ht(model,params):
  random_search = RandomizedSearchCV(model,params,cv=5,scoring='f1')
  random_search.fit(x,y)
  print(model.__class__.__name__)
  print(f"best params : {random_search.best_params_}")
  print(f"best score : {random_search.best_score_}")

In [220]:
lr_hyperparameters = {
        'penalty': ['l2','l1','elasticnet','none'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear'],
        'max_iter': [100, 200, 500]
    }

ht(LogisticRegression(),lr_hyperparameters)

LogisticRegression
best params : {'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 500, 'C': 1}
best score : 0.7911026935258239


In [221]:
knn_hyperparameters = {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski', 'euclidean', 'manhattan']
    }

ht(KNeighborsClassifier(),knn_hyperparameters)

KNeighborsClassifier
best params : {'weights': 'uniform', 'n_neighbors': 11, 'metric': 'manhattan'}
best score : 0.7403155092479523


In [222]:
nb_hyperparameters = {
        'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
    }

ht(GaussianNB(),nb_hyperparameters)

GaussianNB
best params : {'var_smoothing': 1e-08}
best score : 0.7633105108176271


In [223]:
dt_hyperparameters = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 3, 5, 7, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': [None, 'sqrt', 'log2']
    }

ht(DecisionTreeClassifier(),dt_hyperparameters)

DecisionTreeClassifier
best params : {'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 7, 'criterion': 'gini'}
best score : 0.8027252460533987


In [224]:
rf_hyperparameters = {
        'n_estimators': [100, 200, 500],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 3, 5, 7, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    }

ht(RandomForestClassifier(),rf_hyperparameters)

RandomForestClassifier
best params : {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy'}
best score : 0.8125344433468517


In [225]:
gb_hyperparameters = {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 4, 5, 6],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.6, 0.8, 1.0],
        'max_features': ['sqrt', 'log2', None]
    }


ht(GradientBoostingClassifier(),gb_hyperparameters)

GradientBoostingClassifier
best params : {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.05}
best score : 0.8169601020046293


In [226]:
xg_hyperparameters = {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 0.1, 0.2, 0.3],
        'reg_alpha': [0, 0.01, 0.1],
        'reg_lambda': [1, 1.5, 2]
    }


ht(XGBClassifier(),xg_hyperparameters)

XGBClassifier
best params : {'subsample': 0.6, 'reg_lambda': 1.5, 'reg_alpha': 0.1, 'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 1.0}
best score : 0.8153376614684074
